In [5]:
%load_ext autoreload
%autoreload

from os import getcwd
from os.path import join, abspath, pardir, relpath, exists

import requests

from dataclasses import dataclass, field

import dask.dataframe as dd
import pandas as pd
import numpy as np
from numpy import matrixlib as npmat
import networkx as nx
from typing import Union
import pulp as p
from itertools import combinations
from typing import List, Tuple
from enum import Enum
from scipy.stats import kendalltau, spearmanr

from IPython.display import IFrame
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [6]:
parent_dir = abspath(join(getcwd(), pardir))
data_dir = join(parent_dir, 'data')
data_file = join(data_dir, 'products.csv')
parent_dir

'/Users/mohammadzainabbas/Masters/CS/Decision-Modelling/nutri-score'

In [4]:
df = pd.read_csv(data_file)
df.head()

,id,name,image_url,brand,categories,energy,energy_points,energy_value,fiber,fiber_points,...,omega-6-fat_serving,omega-6-fat_unit,omega-6-fat_value,omega-3-fat,omega-3-fat_100g,omega-3-fat_label,omega-3-fat_unit,omega-3-fat_value,omega-6-fat_label,potassium_label
0,3613042717385,Crème de noisette,https://images.openfoodfacts.org/images/produc...,Phil Gourmet,"Aliments et boissons à base de végétaux, Alime...",1368.0,4.0,327.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,8421251894903,Corazones de alcachofa,https://images.openfoodfacts.org/images/produc...,alcachofas,"Alimentos y bebidas de origen vegetal, Aliment...",138.0,0.0,33.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,8410086940022,Espinacas,https://images.openfoodfacts.org/images/produc...,Auchan,"Alimentos y bebidas de origen vegetal, Aliment...",63.0,0.0,15.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,8435307401458,Champiñones enteros,https://images.openfoodfacts.org/images/produc...,Auchan,"Alimentos y bebidas de origen vegetal, Aliment...",92.0,0.0,22.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,3112940769613,Maiz dulce en grano,https://images.openfoodfacts.org/images/produc...,Auchan,"Aliments et boissons à base de végétaux, Alime...",372.0,1.0,89.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [9]:
!wget https://static.openfoodfacts.org/data/en.openfoodfacts.org.products.csv -O ~/Masters/CS/Decision-Modelling/nutri-score/data/raw_products.csv

--2022-11-14 11:05:54--  https://static.openfoodfacts.org/data/en.openfoodfacts.org.products.csv
Resolving static.openfoodfacts.org (static.openfoodfacts.org)... 64:ff9b::d524:fdce, 213.36.253.206
Connecting to static.openfoodfacts.org (static.openfoodfacts.org)|64:ff9b::d524:fdce|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 7325177980 (6.8G) [text/csv]
Saving to: ‘/Users/mohammadzainabbas/Masters/CS/Decision-Modelling/nutri-score/data/raw_products.csv’

s/CS/Decision-Model   1%[                    ] 103.23M   404KB/s    eta 2h 52m ^C


In [11]:
res = requests.get("https://world.openfoodfacts.org/api/v0/product/737628064502.json")
res

<Response [200]>

In [15]:
res = res.json()

In [50]:
def get_mapping(data, key, value):
    values = value.split(".")
    is_all = "all" in values
    _data = data
    for val in values:
        if is_all and val == "all": break
        _data = _data[val]
    return {key: _data} if not is_all else _data

In [53]:
column_mapping = {
    "id": "product.id",
    "name": "product.generic_name_en",
    "extra_labels": "product.labels",
    "image_url": "product.image_url",
    "brand": "product.brand_owner",
    "categories": "product.categories",
    "nutriscore_grade": "product.nutriscore_grade",
    "nutriscore_score": "product.nutriscore_score",
    "nutriscore_data": "product.nutriscore_data.all",
    "nutriments": "product.nutriments.all",
}

In [54]:
data = dict()
for key in column_mapping.keys():
    data.update(get_mapping(res, key, column_mapping[key]))
data

{'id': '737628064502',
 'name': 'Rice Noodles',
 'extra_labels': 'No gluten',
 'image_url': 'https://images.openfoodfacts.org/images/products/073/762/806/4502/front_en.6.400.jpg',
 'brand': 'Simply Asia Foods, Inc.',
 'categories': 'Plant-based foods and beverages, Plant-based foods, Cereals and potatoes, Cereals and their products, Noodles, Rice Noodles',
 'nutriscore_grade': 'c',
 'nutriscore_score': 4,
 'energy': 1611,
 'energy_points': 4,
 'energy_value': 385,
 'fiber': 1.9,
 'fiber_points': 1,
 'fiber_value': 1.9,
 'fruits_vegetables_nuts_colza_walnut_olive_oils': 0,
 'fruits_vegetables_nuts_colza_walnut_olive_oils_points': 0,
 'fruits_vegetables_nuts_colza_walnut_olive_oils_value': 0,
 'grade': 'c',
 'is_beverage': 0,
 'is_cheese': 0,
 'is_fat': 0,
 'is_water': 0,
 'negative_points': 10,
 'positive_points': 6,
 'proteins': 9.62,
 'proteins_points': 5,
 'proteins_value': 9.62,
 'saturated_fat': 1.92,
 'saturated_fat_points': 1,
 'saturated_fat_ratio': 24.9674902470741,
 'saturated

In [20]:
res["product"]["brand_owner"]

'Simply Asia Foods, Inc.'

#### From static data

In [5]:
# df = pd.read_csv(join(data_dir, "raw_products.csv"), chunksize=10 ** 4)
# df.head()

d = pd.read_csv(join(data_dir, "raw_products.csv"), nrows=10 ** 3)

ParserError: Error tokenizing data. C error: Expected 37 fields in line 3, saw 52


In [2]:
type(d)

NameError: name 'd' is not defined